# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_df = pd.read_csv('city_data.csv')
cities_df.dropna(inplace = True) 
cities_df.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,East London,-33.0153,27.9116,18.17,73,68,6.01,ZA,1636849728
1,Hambantota,6.1241,81.1185,25.63,83,96,5.74,LK,1636849729
2,Ushuaia,-54.8000,-68.3000,11.81,43,40,4.63,AR,1636849715
3,Vaini,-21.2000,-175.2000,24.19,69,47,3.74,TO,1636849731
4,Pagudpud,18.5616,120.7868,27.98,87,84,6.06,PH,1636849732


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
Humidity = cities_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:

#   * A max temperature lower than 80 degrees but higher than 70.

#   * Wind speed less than 10 mph.

#   * Zero cloudiness.

#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ideal_df = cities_df

ideal_df = ideal_df.loc[cities_df["Max temp"] < 27] 

ideal_df = ideal_df.loc[cities_df["Max temp"] > 21] 

ideal_df = ideal_df.loc[cities_df["Wind Speed"] < 10]

ideal_df = ideal_df.loc[cities_df["Cloudiness"] == 0]

# ideal_df = ideal_df.dropna

ideal_df.head(20)

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,Carnarvon,-24.8667,113.6333,23.04,68,0,5.66,AU,1636849775
102,Mount Isa,-20.7333,139.5000,22.87,25,0,8.75,AU,1636849802
137,Aswān,24.0934,32.9070,22.74,71,0,3.30,EG,1636849904
202,Hervey Bay,-25.2986,152.8535,26.06,36,0,1.78,AU,1636849956
218,Zhuhai,22.2769,113.5678,21.14,60,0,5.66,CN,1636849968
221,Kibala,9.1104,18.3463,25.97,69,0,4.12,TD,1636849970
223,Yeppoon,-23.1333,150.7333,26.88,43,0,3.35,AU,1636849972
304,Sur,22.5667,59.5289,23.83,34,0,1.73,OM,1636850033
461,Emerald,-23.5333,148.1667,25.19,27,0,2.57,AU,1636850151
483,San Lorenzo,-25.3333,-57.5333,26.52,57,0,3.13,PY,1636850169


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 63: Carnarvon.
Closest hotel is Hospitality Carnarvon.
------------
Retrieving Results for Index 102: Mount Isa.
Closest hotel is ibis Styles Mt Isa Verona.
------------
Retrieving Results for Index 137: Aswān.
Closest hotel is Nuba Nile.
------------
Retrieving Results for Index 202: Hervey Bay.
Closest hotel is Shelly Bay Resort.
------------
Retrieving Results for Index 218: Zhuhai.
Closest hotel is Nanyang Seascape Hotel Zhuhai Hotel.
------------
Retrieving Results for Index 221: Kibala.
Closest hotel is hotel safari.
------------
Retrieving Results for Index 223: Yeppoon.
Closest hotel is Villa Mar Colina.
------------
Retrieving Results for Index 304: Sur.
Closest hotel is Sur Plaza Hotel.
------------
Retrieving Results for Index 461: Emerald.
Closest hotel is The Irish Village.
------------
Retrieving Results for Index 483: San Lorenzo.
Closest hotel is Hotel del Sur.
------------
Retrieving Results for Index 492: Salalah.
Closest hotel is HAMDAN P

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))